In [1]:
# TODO implementar os demais agentes como partes do cérebro
# TODO implementar memória de conversa postgres
# TODO implementar memória da amygdala
# TODO verificar se algum outro ponto deve ter memória

In [2]:
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

from typing import Literal

from agents.prefrontal_cortex import prefrontal_cortex_agent, prefrontal_cortex_out_agent
from agents.behavior_planner import behavior_planner_agent
from agents.decision_maker import decision_maker_agent
from agents.social_behavior_modulator import social_behavior_modulator_agent
from agents.complex_thought_planner import complex_thought_planner_agent
from agents.personality_expression_planner import personality_expression_planner_agent
from agents.models.brain import AgentState


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

## Create tools

In [4]:
search = GoogleSerperAPIWrapper(gl='br', hl='pt-BR', k=3)


@tool
def serper_tool(question: Literal["nyc", "sf"]):
    """Useful for when you need to ask with search"""
    return search.run(question)
    

python_repl_tool = PythonREPLTool()

### Models

In [5]:
workflow = StateGraph(AgentState)

workflow.add_node("prefrontal_cortex_agent", prefrontal_cortex_agent)
workflow.add_node("behavior_planner_agent", behavior_planner_agent)
workflow.add_node("decision_maker_agent", decision_maker_agent)
workflow.add_node("social_behavior_modulator_agent", social_behavior_modulator_agent)
workflow.add_node("complex_thought_planner_agent", complex_thought_planner_agent)
workflow.add_node("personality_expression_planner_agent", personality_expression_planner_agent)
workflow.add_node("prefrontal_cortex_out_agent", prefrontal_cortex_out_agent)

workflow.set_entry_point("prefrontal_cortex_agent")
workflow.add_edge("prefrontal_cortex_agent", "behavior_planner_agent")
workflow.add_edge("prefrontal_cortex_agent", "decision_maker_agent")
workflow.add_edge("prefrontal_cortex_agent", "social_behavior_modulator_agent")
workflow.add_edge("prefrontal_cortex_agent", "complex_thought_planner_agent")
workflow.add_edge("prefrontal_cortex_agent", "personality_expression_planner_agent")
workflow.add_edge(["behavior_planner_agent","decision_maker_agent", "social_behavior_modulator_agent", "complex_thought_planner_agent", "personality_expression_planner_agent"], "prefrontal_cortex_out_agent")
workflow.add_edge("prefrontal_cortex_out_agent", END)

#memory = MemorySaver()
#graph = workflow.compile(checkpointer=memory)

In [6]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [7]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}
inputs = {"messages": [
        HumanMessage(content="Oi, meu nome é Cristian!")
    ]
}

from psycopg_pool import AsyncConnectionPool
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver

connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

import os
DB_URI = os.getenv("DB_URI")

async with AsyncConnectionPool(
    # Example configuration
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = AsyncPostgresSaver(pool)

    # NOTE: you need to call .setup() the first time you're using your checkpointer
    #await checkpointer.setup()

    graph = workflow.compile(checkpointer=checkpointer)

    
    async for chunk in graph.astream(inputs, stream_mode="values", config=config):
        chunk["messages"][-1].pretty_print()
    


error connecting in 'pool-1': connection failed: connection to server at "127.0.0.1", port 30432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
error connecting in 'pool-1': connection failed: connection to server at "127.0.0.1", port 30432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
error connecting in 'pool-1': connection failed: connection to server at "127.0.0.1", port 30432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
error connecting in 'pool-1': connection failed: connection to server at "127.0.0.1", port 30432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
error connecting in 'pool-1': connection failed: connection to server at "127.0.0.1", port 30432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
error connecting in 'pool-1': 

CancelledError: 

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}

for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Oi, meu nome é Cristian!")
        ]
    },
    config
):
    if "__end__" not in s:
        print(s)
        print("----")


In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}

for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Qual meu nome?")
        ]
    },
    config
):
    if "__end__" not in s:
        print(s)
        print("----")


## Invoke the team

With the graph created, we can now invoke it and see how it performs!


In [ ]:
from langchain_core.messages import HumanMessage

for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Estou com alguns problemas, você pode me ajudar?")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

In [ ]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Estou com alguns problemas, você pode me ajudar?")]},
    {"recursion_limit": 5},
):
    if "__end__" not in s:
        print(s)
        print("----")

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "Estou com alguns problemas, você pode me ajudar?")]}

print_stream(graph.stream(inputs, stream_mode="values"))